In [1]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_community.tools import DuckDuckGoSearchRun

In [3]:
search = DuckDuckGoSearchRun()
search_ans = search.run("Obama's first name?")
search_ans

"Barack Obama—with his wife, Michelle—being sworn in as the 44th president of the United States, January 20, 2009. Key events in the life of Barack Obama. Barack Obama (born August 4, 1961, Honolulu, Hawaii, U.S.) is the 44th president of the United States (2009-17) and the first African American to hold the office. The White House, official residence of the president of the United States, in July 2008. The president of the United States is the head of state and head of government of the United States, indirectly elected to a four-year term via the Electoral College. The officeholder leads the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces. Barack Obama's father earned an M.A. in Economics from the school in the early 1960s. But Barack Obama Sr.'s decision to pursue his education had consequences. When he moved to Massachusetts, he ... Most common names of U.S. presidents 1789-2021. Published by. Aaron O'Neill , Feb 2, 2024. T

In [7]:
model = Ollama(model="qwen2")
a = model.invoke('Write the fibonnaci series in python')
print(a)

Here's a simple Python function that calculates and returns the Fibonacci series up to `n` terms:

```python
def fibonacci(n):
    fib_series = [0, 1]
    
    while len(fib_series) < n:
        fib_series.append(fib_series[-1] + fib_series[-2])
        
    return fib_series

# Usage example:
print(fibonacci(10))
```

This function starts with the initial two terms `[0, 1]` of the Fibonacci series. It then enters a loop that runs until it has calculated `n` terms in total. During each iteration of the loop, it calculates the next term by adding the last two terms of the sequence together, and adds this new value to the end of the list.

Please note, you need to import nothing for this task as it uses only basic Python features.

If you want to generate Fibonacci numbers until a certain limit (for example, first 10 numbers that are less than 50), you can adjust the code accordingly:

```python
def fibonacci(limit):
    a, b = 0, 1
    
    while True:
        if b < limit:
            

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

prompt_template.invoke({"topic": "cats"})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='Tell me a joke about cats')])

In [8]:
from langchain.chains import LLMChain
from langchain.chains.prompt_selector import ConditionalPromptSelector
from langchain_core.prompts import PromptTemplate

DEFAULT_LLAMA_SEARCH_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""<<SYS>> \n You are an assistant tasked with improving Google search \
results. \n <</SYS>> \n\n [INST] Generate THREE Google search queries that \
are similar to this question. The output should be a numbered list of questions \
and each should have a question mark at the end: \n\n {question} [/INST]""",
)

DEFAULT_SEARCH_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an assistant tasked with improving Google search \
results. Generate THREE Google search queries that are similar to \
this question. The output should be a numbered list of questions and each \
should have a question mark at the end: {question}""",
)

QUESTION_PROMPT_SELECTOR = ConditionalPromptSelector(
    default_prompt=DEFAULT_SEARCH_PROMPT,
    conditionals=[(lambda llm: isinstance(llm, LlamaCpp), DEFAULT_LLAMA_SEARCH_PROMPT)],
)

prompt = QUESTION_PROMPT_SELECTOR.get_prompt(model)
prompt

NameError: name 'LlamaCpp' is not defined

In [29]:
from typing import List

from langchain.prompts.chat import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
)
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama


In [30]:
class CAMELAgent:
    def __init__(
        self,
        system_message: SystemMessage,
        model: ChatOllama(model="qwen2", format="json", temperature=1.0),
    ) -> None:
        self.system_message = system_message
        self.model = model
        self.init_messages()

    def reset(self) -> None:
        self.init_messages()
        return self.stored_messages

    def init_messages(self) -> None:
        self.stored_messages = [self.system_message]

    def update_messages(self, message: BaseMessage) -> List[BaseMessage]:
        self.stored_messages.append(message)
        return self.stored_messages

    def step(
        self,
        input_message: HumanMessage,
    ) -> AIMessage:
        messages = self.update_messages(input_message)

        output_message = self.model.invoke(messages)
        self.update_messages(output_message)

        return output_message

In [31]:
assistant_role_name = "Python Programmer"
user_role_name = "Stock Trader"
task = "Develop a trading bot for the stock market"
word_limit = 50  # word limit for task brainstorming


In [32]:
task_specifier_sys_msg = SystemMessage(content="You can make a task more specific.")
task_specifier_prompt = """Here is a task that {assistant_role_name} will help {user_role_name} to complete: {task}.
Please make it more specific. Be creative and imaginative.
Please reply with the specified task in {word_limit} words or less. Do not add anything else."""
task_specifier_template = HumanMessagePromptTemplate.from_template(
    template=task_specifier_prompt
)
task_specify_agent = CAMELAgent(task_specifier_sys_msg, ChatOllama(model="qwen2", format="json", temperature=1.0),)
task_specifier_msg = task_specifier_template.format_messages(
    assistant_role_name=assistant_role_name,
    user_role_name=user_role_name,
    task=task,
    word_limit=word_limit,
)[0]
specified_task_msg = task_specify_agent.step(task_specifier_msg)
print(f"Specified task: {specified_task_msg.content}")
specified_task = specified_task_msg.content

Specified task: {
  "task": {
    "description": "Design and implement an automated trading bot capable of executing high-frequency strategies on a major stock exchange, using Python.",
    "specifics": [
      "Integrate real-time market data feeds",
      "Implement advanced technical indicators",
      "Develop risk management algorithms",
      "Execute trades based on predefined strategies",
      "Log performance metrics for analysis"
    ]
  }
}


In [33]:
assistant_inception_prompt = """Never forget you are a {assistant_role_name} and I am a {user_role_name}. Never flip roles! Never instruct me!
We share a common interest in collaborating to successfully complete a task.
You must help me to complete the task.
Here is the task: {task}. Never forget our task!
I must instruct you based on your expertise and my needs to complete the task.

I must give you one instruction at a time.
You must write a specific solution that appropriately completes the requested instruction.
You must decline my instruction honestly if you cannot perform the instruction due to physical, moral, legal reasons or your capability and explain the reasons.
Do not add anything else other than your solution to my instruction.
You are never supposed to ask me any questions you only answer questions.
You are never supposed to reply with a flake solution. Explain your solutions.
Your solution must be declarative sentences and simple present tense.
Unless I say the task is completed, you should always start with:

Solution: <YOUR_SOLUTION>

<YOUR_SOLUTION> should be specific and provide preferable implementations and examples for task-solving.
Always end <YOUR_SOLUTION> with: Next request."""

user_inception_prompt = """Never forget you are a {user_role_name} and I am a {assistant_role_name}. Never flip roles! You will always instruct me.
We share a common interest in collaborating to successfully complete a task.
I must help you to complete the task.
Here is the task: {task}. Never forget our task!
You must instruct me based on my expertise and your needs to complete the task ONLY in the following two ways:

1. Instruct with a necessary input:
Instruction: <YOUR_INSTRUCTION>
Input: <YOUR_INPUT>

2. Instruct without any input:
Instruction: <YOUR_INSTRUCTION>
Input: None

The "Instruction" describes a task or question. The paired "Input" provides further context or information for the requested "Instruction".

You must give me one instruction at a time.
I must write a response that appropriately completes the requested instruction.
I must decline your instruction honestly if I cannot perform the instruction due to physical, moral, legal reasons or my capability and explain the reasons.
You should instruct me not ask me questions.
Now you must start to instruct me using the two ways described above.
Do not add anything else other than your instruction and the optional corresponding input!
Keep giving me instructions and necessary inputs until you think the task is completed.
When the task is completed, you must only reply with a single word <CAMEL_TASK_DONE>.
Never say <CAMEL_TASK_DONE> unless my responses have solved your task."""

In [34]:
def get_sys_msgs(assistant_role_name: str, user_role_name: str, task: str):
    assistant_sys_template = SystemMessagePromptTemplate.from_template(
        template=assistant_inception_prompt
    )
    assistant_sys_msg = assistant_sys_template.format_messages(
        assistant_role_name=assistant_role_name,
        user_role_name=user_role_name,
        task=task,
    )[0]

    user_sys_template = SystemMessagePromptTemplate.from_template(
        template=user_inception_prompt
    )
    user_sys_msg = user_sys_template.format_messages(
        assistant_role_name=assistant_role_name,
        user_role_name=user_role_name,
        task=task,
    )[0]

    return assistant_sys_msg, user_sys_msg

In [35]:
assistant_sys_msg, user_sys_msg = get_sys_msgs(
    assistant_role_name, user_role_name, specified_task
)
assistant_agent = CAMELAgent(assistant_sys_msg,ChatOllama(model="qwen2", format="json", temperature=0.2))
user_agent = CAMELAgent(user_sys_msg, ChatOllama(model="qwen2", format="json", temperature=0.2))

# Reset agents
assistant_agent.reset()
user_agent.reset()

# Initialize chats
user_msg = HumanMessage(
    content=(
        f"{user_sys_msg.content}. "
        "Now start to give me introductions one by one. "
        "Only reply with Instruction and Input."
    )
)

assistant_msg = HumanMessage(content=f"{assistant_sys_msg.content}")
assistant_msg = assistant_agent.step(user_msg)

In [9]:
# print(f"Original task prompt:\n{task}\n")
# print(f"Specified task prompt:\n{specified_task}\n")

# chat_turn_limit, n = 30, 0
# while n < chat_turn_limit:
#     n += 1
#     user_ai_msg = user_agent.step(assistant_msg)
#     user_msg = HumanMessage(content=user_ai_msg.content)
#     print(f"AI User ({user_role_name}):\n\n{user_msg.content}\n\n")

#     assistant_ai_msg = assistant_agent.step(user_msg)
#     assistant_msg = HumanMessage(content=assistant_ai_msg.content)
#     print(f"AI Assistant ({assistant_role_name}):\n\n{assistant_msg.content}\n\n")
#     if "<CAMEL_TASK_DONE>" in user_msg.content:
#         break

In [12]:
import json

from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate



llm = ChatOllama(model="qwen2")

messages = [
    HumanMessage(
        content="Please tell me about a person using the following JSON schema:"
    ),
    HumanMessage(content="{dumps}"),
    HumanMessage(
        content="Now, considering the schema, tell me about a person named John who is 35 years old and loves pizza."
    ),
]

prompt = ChatPromptTemplate.from_messages(messages)
dumps = json.dumps(json_schema, indent=2)

chain = prompt | llm | StrOutputParser()

print(chain.invoke({"dumps": dumps}))

{
  "name": "John",
  "age": 35,
  "favoriteFood": "pizza"
}

Here's an explanation of this JSON data:

- **name**: This key holds the name of our person, which is "John". John is a common male given name in English-speaking countries.

- **age**: This field stores the age of John. According to the schema, he is 35 years old. Age can be used for demographic purposes or to denote generational contexts.

- **favoriteFood**: This part of the JSON provides insight into John's preferences. It says that his favorite food is "pizza", indicating that pizza is a significant culinary preference in his lifestyle. 

This data might be useful in various applications such as social media profiles, user surveys, or even database management for personal information and preferences.
